In [115]:
import pandas as pd
import numpy as np
import re
import string
import joblib
import datetime as dt
import html
from datetime import datetime

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import emoji



In [116]:
aapl_df = pd.read_csv('Stock_Data/AAPL.csv')
tsla_df = pd.read_csv('Stock_Data/TSLA.csv')

# Other technology stocks:
goog_df = pd.read_csv('Stock_Data/GOOG.csv')
amzn_df = pd.read_csv('Stock_Data/AMZN.csv')
nflx_df = pd.read_csv('Stock_Data/NFLX.csv')
shop_df = pd.read_csv('Stock_Data/SHOP.csv')
pltr_df = pd.read_csv('Stock_Data/PLTR.csv')

# Banks
jpm_df = pd.read_csv('Stock_Data/JPM.csv')
c_df = pd.read_csv('Stock_Data/C.csv')

aapl_df['stock'] = 'AAPL'
tsla_df['stock'] = 'TSLA'
goog_df['stock'] = 'GOOG'
amzn_df['stock'] = 'AMZN'
nflx_df['stock'] = 'NFLX'
shop_df['stock'] = 'SHOP'
pltr_df['stock'] = 'PLTR'
jpm_df['stock'] = 'JPM'
c_df['stock'] = 'C'

df = pd.concat([aapl_df, tsla_df, goog_df, amzn_df,
                nflx_df, shop_df, pltr_df, jpm_df, c_df], ignore_index=True, sort=False)

df.columns

Index(['User', 'Body', 'Created At', 'Number of Likes', 'Entities', 'stock',
       'Unnamed: 0', 'Created  At', 'Number  of  Likes'],
      dtype='object')

In [117]:
len(df)

2678910

In [118]:
df = df.rename(columns={'User':'User',
        'Body': 'Body',
        'Created At' : 'Created At',
        'Number of Likes' : 'Number of Likes',
        'Entities':'Entities',
        'stock' : 'stock',
       'Unnamed: 0' : 'WhatIsThis',
       'Created  At' : 'Created At',
       'Number  of  Likes' : 'Number of Likes'})

df = df.groupby(df.columns, axis=1).sum()
df = df.drop(['WhatIsThis'], axis=1)

df.columns

Index(['Body', 'Created At', 'Entities', 'Number of Likes', 'User', 'stock'], dtype='object')

In [119]:
df['Created At'] = pd.to_datetime(df['Created At']).dt.tz_localize(None)

df['date'] = df['Created At'].dt.date
df['time'] = df['Created At'].dt.time

In [120]:
def our_date_range(df, col):
    df = df[df[col] >= '2022-1-1']
    df = df[df[col] < '2023-10-1']
    return df

df = our_date_range(df, 'Created At')
df = df.drop_duplicates()
len(df)

2612166

In [121]:
df['Entities'].value_counts()

Entities
0          1336381
Bullish     862999
Bearish     412786
Name: count, dtype: int64

In [122]:
df['Body'].isna().sum()

0

In [1]:
# 1336381 / 2612166

0.5115988034451103

In [123]:
df.sample(20)

,Body,Created At,Entities,Number of Likes,User,stock,date,time
886058,$TSLA \nToday: Cathie wood buying + dumb retai...,2023-01-04 23:09:18,Bearish,1.0,4704216,TSLA,2023-01-04,23:09:18
672824,$TSLA block the bears!!,2023-03-13 15:06:59,Bullish,0.0,6877415,TSLA,2023-03-13,15:06:59
959414,$TSLA what a fucking manipulated market unbeli...,2022-12-22 20:36:13,0,3.0,675289,TSLA,2022-12-22,20:36:13
120311,$AAPL \nDo not fight the trend!,2022-10-07 19:29:41,Bearish,1.0,2059008,AAPL,2022-10-07,19:29:41
428267,$TSLA hahhahha 🐻,2023-07-19 20:09:53,Bullish,0.0,4723369,TSLA,2023-07-19,20:09:53
988254,$TSLA confidence has completely eroded,2022-12-19 16:07:56,Bullish,0.0,6332023,TSLA,2022-12-19,16:07:56
2643329,$JPM anti China racists,2022-08-29 19:45:07,0,0.0,1954748,JPM,2022-08-29,19:45:07
2199297,$NFLX missed (an already super low) earnings e...,2023-01-19 21:11:59,0,1.0,5727384,NFLX,2023-01-19,21:11:59
1450843,$TSLA musk should invent a 1000 mile long exte...,2022-05-10 16:56:31,Bearish,0.0,1966560,TSLA,2022-05-10,16:56:31
745711,$TSLA,2023-02-15 10:24:37,0,1.0,3981397,TSLA,2023-02-15,10:24:37


In [124]:
# Dropping all emoji
def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"
                               u"\U0001F300-\U0001F5FF"
                               u"\U0001F680-\U0001F6FF"
                               u"\U0001F1E0-\U0001F1FF"
                               u"\U00002500-\U00002BEF"
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [125]:
def preprocessing(raw_df):
    remove_at_symbol = r'[@¿¡¤¦§¨ª«»¬¯¹²³¶µ·¼½¾¿ßàáâãäåæçèéêëìíîïðñòóôõö÷øùúûüýþÿāăćĉčđęğģĥĺōőœśşšţūŵžƒơưǂșɓəɠɢɪɴɾʍʖ#]\w+\s*'
    raw_df['Body'] = raw_df['Body'].str.replace(remove_at_symbol, '', regex=True)
    raw_df['Body'] = raw_df['Body'].str.lower()

    raw_df['Body'] = raw_df['Body'].str.replace(r'\$(?![0-9])', 'dollar_', regex=True)

    url_remove = r'https?://\S+|www\.\S+'
    raw_df['Body'] = raw_df['Body'].str.replace(url_remove, '', regex=True)

    line_break_remove = r'(\n|\r)'
    raw_df['Body'] = raw_df['Body'].str.replace(line_break_remove, '', regex=True)

    #Replacing excessive spaces
    raw_df['Body'] = raw_df['Body'].str.strip().str.replace(r'\s+', ' ', regex=True)

    raw_df['Body'] = raw_df['Body'].astype(str)

    # Remove all emoji
    raw_df['Body'] = raw_df['Body'].apply(lambda text: remove_emoji(text=text))
    # raw_df['Body'] = raw_df['Body'].apply(lambda x: emoji.demojize(x))            # This line was converting emojis into text
    raw_df['Body'] = raw_df['Body'].replace(r'::', ", ", regex=True)
    raw_df['Body'] = raw_df['Body'].replace(r':', "", regex=True)
    raw_df['Body'] = raw_df['Body'].apply(lambda x: x[1:] if x.startswith(",") else x)
    raw_df['Body'] = raw_df['Body'].apply(lambda x: x[1:] if x.startswith(('$', '#', '-')) else x)

    return raw_df

data_df = preprocessing(df)

In [126]:
new_df = data_df[(data_df['Body'] != "0") | (data_df['Entities'] != "0")]
new_df = new_df[(new_df['Body'] != "j") | (new_df['Entities'] != "0")]
new_df = new_df[(new_df['Body'] != "0") | (new_df['Entities'] != "0")]
new_df = new_df[(new_df['Body'] != "3") | (new_df['Entities'] != "0")]
new_df = new_df[(new_df['Body'] != "dollar_c") | (new_df['Entities'] != "0")]
new_df = new_df[(new_df['Body'] != "dollar_c ") | (new_df['Entities'] != "0")]
new_df = new_df[(new_df['Body'] != "dollar_c.") | (new_df['Entities'] != "0")]

output2.csv should be 2612035 in length!

In [127]:
len(new_df)

2612166

In [128]:
# 2612166 - 2612035

131

Somehow we are missing 131 values!

In [129]:
new_df.sample(24)

,Body,Created At,Entities,Number of Likes,User,stock,date,time
552566,dollar_tsla this poor lost soul got triggered ...,2023-05-24 19:15:15,0,0.0,6850228,TSLA,2023-05-24,19:15:15
1667071,"dollar_tsla come on people, you know damn well...",2022-01-26 21:29:18,0,0.0,5281609,TSLA,2022-01-26,21:29:18
2630478,dollar_spy dollar_xlf dollar_bac dollar_jpm do...,2023-03-15 10:37:55,0,3.0,2016285,JPM,2023-03-15,10:37:55
579608,dollar_tsla april vehicle registration for tes...,2023-05-05 12:59:07,Bullish,3.0,4150042,TSLA,2023-05-05,12:59:07
2365242,dollar_shop volume going to be crazy with buye...,2022-05-05 13:18:23,0,1.0,4091527,SHOP,2022-05-05,13:18:23
1855070,dollar_spy dollar_aapl dollar_qqq gunna get ga...,2023-05-04 20:35:03,Bearish,2.0,2063593,AMZN,2023-05-04,20:35:03
2606100,dollar_pltr it may be a long time…maybe 18 mon...,2022-01-09 16:24:01,0,6.0,5888028,PLTR,2022-01-09,16:24:01
1275657,dollar_tsla so gotta get my ass ready for anot...,2022-08-07 21:02:56,0,1.0,1535370,TSLA,2022-08-07,21:02:56
2166826,dollar_nflx yep those worth zero now . be smar...,2023-07-14 18:17:00,0,0.0,213680,NFLX,2023-07-14,18:17:00
2210274,dollar_spy dollar_qqq dollar_nflx my followers...,2022-11-30 17:27:34,Bearish,0.0,6600525,NFLX,2022-11-30,17:27:34
